In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/test-val/test_data.tsv
/kaggle/input/test-val/val_data.tsv
/kaggle/input/ultimate-test-manual-check/test_ult.tsv
/kaggle/input/bf-model-dev-copy/config.json
/kaggle/input/bf-model-dev-copy/tokenizer.json
/kaggle/input/bf-model-dev-copy/tf_model.h5
/kaggle/input/bf-model-dev-copy/tokenizer_config.json
/kaggle/input/bf-model-dev-copy/tf_model.preproc
/kaggle/input/bf-model-dev-copy/special_tokens_map.json
/kaggle/input/bf-model-dev-copy/vocab.txt


In [2]:
!pip install -q ktrain
!pip install -q cached-path

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.2 which is incompatible.
pdpbox 0.2.1 requires matplotlib==3.1.1, but you have matplotlib 3.5.1 which is incompatible.
imbalanced-learn 0.9.0 requires scikit-learn>=1.0.1, but you have scikit-learn 0.24.2 which is incompatible.
featuretools 1.6.0 requires numpy>=1.21.0, but you have numpy 1.20.3 which is incompatible.


In [3]:
import tensorflow as tf
import ktrain 
from cached_path import cached_path


# Activate AMP
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

# Load model (method 1)
#predictor=ktrain.load_predictor('../input/bf-model-dev-copy')

# Load model (method 2)
model_path = cached_path("https://github.com/PathwayCommons/pathway-abstract-classifier/releases/download/pretrained-models/title_abstract_model.zip", extract_archive=True)
predictor = ktrain.load_predictor(model_path)

downloading:   0%|          | 0.00/389M [00:00<?, ?iB/s]

In [4]:
# Read in validation data
df=pd.read_csv('../input/test-val/val_data.tsv', delimiter="\t")

In [5]:
#pre-process data 
df['class']=df['class'].astype('bool')
titles=df['title'].tolist()
abstracts=df['abstract'].tolist()
texts = [title + " [SEP] " + abstract for title, abstract in zip(titles, abstracts)]
y_val=df['class'].to_numpy()

In [6]:
# Check performance on validation set
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import matthews_corrcoef


predictions=predictor.predict((texts))
print(classification_report((y_val), predictions))
print(matthews_corrcoef((y_val), predictions))

              precision    recall  f1-score   support

       False       0.94      0.95      0.95       842
        True       0.77      0.77      0.77       200

    accuracy                           0.91      1042
   macro avg       0.86      0.86      0.86      1042
weighted avg       0.91      0.91      0.91      1042

0.7142926808037249


In [7]:
# Read in test data 
df_test=pd.read_csv('../input/test-val/test_data.tsv', delimiter="\t")

In [8]:
#pre-process data 
df_test['class']=df_test['class'].astype('bool')
titles=df_test['title'].tolist()
abstracts=df_test['abstract'].tolist()
texts = [title + " [SEP] " + abstract for title, abstract in zip(titles, abstracts)]
y_test=df_test['class'].to_numpy()

In [9]:
# Check performance on test set
predictions=predictor.predict((texts))
print(classification_report((y_test), predictions))
print(matthews_corrcoef((y_test), predictions))

              precision    recall  f1-score   support

       False       0.93      0.95      0.94       485
        True       0.79      0.71      0.75       115

    accuracy                           0.91       600
   macro avg       0.86      0.83      0.85       600
weighted avg       0.91      0.91      0.91       600

0.6942763553429271


In [10]:
# Read in test data to be manually checked
df_ultest=pd.read_csv('../input/ultimate-test-manual-check/test_ult.tsv', delimiter ="\t")
df_ultest.replace(np.nan, '', regex=True, inplace=True)
print(df_ultest)

           uid                                           abstract  \
0     30509971                                                      
1     30509970                                                      
2     30509969                                                      
3     30478191  Meiotic resumption-coupled degradation of mate...   
4     30470721                                                      
...        ...                                                ...   
1004  25388951  Transposable elements (TEs) generate mutations...   
1005  25385835  Formation of a new centriole adjacent to a pre...   
1006  25378477  The Bcl-2 proteins Bax and Bak can permeabiliz...   
1007  25371407  The final event of the eukaryotic cell cycle i...   
1008  25361606  The Unfolded Protein Response (UPR) indirectly...   

                                                  title           journal  
0                          Open Access-or Open Science?  The EMBO journal  
1     More than jus

In [11]:
#Pre-Process input
titles=df_ultest['title'].tolist()
abstracts=df_ultest['abstract'].tolist()
texts = [title + " [SEP] " + abstract for title, abstract in zip(titles, abstracts)]

In [12]:
# Make predictions (with probs) on input and add to dataframe
predictions=predictor.predict((texts))
probs= predictor.predict_proba((texts))
df_ultest['Predictions']=predictions
df_ultest['Probs']=probs[:,1]

In [13]:
df_ultest.to_csv('manual_test.tsv', sep="\t", index= False)